In [216]:
# ライブラリの読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import seaborn as sns
import time
import torch
import torch.nn as nn
import torch.optim as optimizers
from scipy.stats import norm
from numpy.random import *
from scipy import optimize

np.random.seed(9837)
torch.manual_seed(9837)
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 100)

In [2]:
# 多項分布の乱数を生成する関数
def rmnom(pr, n, k, pattern):
    if pattern==1:
        z_id = np.array(np.argmax(np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis], axis=1), dtype="int")
        Z = np.diag(np.repeat(1, k))[z_id, ]
        return z_id, Z
    z_id = np.array(np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1), dtype="int")
    return z_id

# ディリクリ分布の乱数を生成する関数
def Dirichlet(alpha, n):
    x = torch.Tensor(np.random.dirichlet(alpha, n))
    return x

# 入力データの定義

In [3]:
# データの設定
types = 2
min_word = 5
max_word = 50
k11 = 5   # topic wordのsyntax数
k12 = 7   # general wordのsyntax数
k1 = k11 + k12   # syntax数
k21 = 15   # topic wordのトピック数
k22 = 10   # general wordのトピック数
k2 = k21 + k22   # topic数
k3 = 15   # word classのトピック数
d = 5000   # 文書数
v11 = 1000  # topic wordのvocabulary数
v12 = 250   # general wordのvocabulary数
v1 = v11 + v12   # vocabulary数
v2 = 100   # word class数
pt = np.random.poisson(np.random.gamma(12.5, 1.0, d), d)
pt[pt < 5] = 5
M = np.sum(pt)
w = np.random.poisson(np.random.gamma(17.5, 1.5, np.sum(pt)), np.sum(pt))
w[w < min_word] = min_word
N = np.sum(w)

# データベクトルを定義
k_vec1 = np.repeat(1.0, k1)
k_vec21 = np.repeat(1.0, k21)
k_vec22 = np.repeat(1.0, k22)
index_k11 = np.arange(k11)
index_k12 = np.arange(k12) + k11
index_k21 = np.arange(k21)
index_k22 = np.arange(k22) + k21
index_v11 = np.arange(v11)
index_v12 = np.arange(v12) + v11

In [25]:
# IDとインデックスを定義
# IDの定義
m = np.repeat(0, d)
doc_list = []
d_id = []
doc_id = np.repeat(np.arange(d), pt)
for i in range(d):
    doc_list.append(np.where(doc_id==i)[0].astype("int"))
    m[i] = np.sum(w[doc_list[i]])
    d_id.append(np.repeat(i, m[i]))
d_id = np.hstack((d_id))
sentence_id = np.repeat(np.arange(M), w)
pt_id = np.hstack(([np.arange(w[i]) for i in range(M)]))

# 文書のインデックスを定義
d_list = []
sentence_list = []
for i in range(d):
    d_list.append(np.where(d_id==i)[0].astype("int"))
for i in range(M):
    sentence_list.append(np.where(sentence_id==i)[0].astype("int"))
    
# 語順のインデックスを定義
max_pt = np.max(pt_id) + 1
pt_list = []
pt_n = np.repeat(0, max_pt)
for j in range(max_pt):
    pt_list.append(np.array(np.where(pt_id==j)[0], dtype="int"))
    pt_n[j] = pt_list[j].shape[0]

In [246]:
# 事前分布の定義
# HMMの事前分布を定義
alpha1 = np.repeat(1.0, k1)
alpha2 = np.append(np.repeat(0.5, k1), 5.0)

# トピック分布の事前分布
beta1 = np.repeat(0.2, k21)
beta2 = np.repeat(0.25, k22)

# 単語分布の事前分布
max_word = 30
gamma11 = np.full((k1, v11), 0.0025)
gamma12 = np.full((k1, v12), 0.0025)
gamma11[index_k11, ] = 0.05
gamma12[index_k12, ] = 0.05
gamma1 = np.hstack((gamma11, gamma12))
gamma21 = np.full((k2, v11), 0.001)
gamma22 = np.full((k2, v12), 0.001)
gamma21[index_k21, ] = 0.025
gamma22[index_k22, ] = 0.025
gamma2 = np.hstack((gamma21, gamma22))

In [247]:
# パラメータを生成
# HMMの推移確率を生成
pi1 = np.append(np.random.dirichlet(alpha1, 1), 0.0).reshape(-1)
pi2 = np.random.dirichlet(alpha2, k1+1)

# ディリクリ分布からトピック分布を生成
kappa = np.random.normal(0, 0.75, v1)
theta1 = np.random.dirichlet(beta1, v1)
theta2 = np.random.dirichlet(beta2, v2)

# 単語分布の事前分布
psi = np.array([np.random.dirichlet(gamma1[j, ], 1).reshape(-1) for j in range(k1)])
phi = np.array([np.random.dirichlet(gamma2[j, ], 1).reshape(-1) for j in range(k2)])

In [248]:
# 応答変数を生成
# 生成したデータの格納用配列
S = np.zeros((N, k1+1), dtype="int")
s = np.repeat(0, N)
Zi = np.zeros((N, k2), dtype="int")
z = np.repeat(-1, N)
word_id = np.repeat(0, N)
word_long = np.full((N, max_pt), -1)
attention_id = np.repeat(-1, N)

# トピックと単語を生成
for j in range(max_pt):

    # 語順に応じた生成を実行
    if j==0:
        
        # 語順が先頭の単語を生成
        # 多項分布からsyntaxを生成
        index = pt_list[j]
        S[index, ] = np.random.multinomial(1, pi1, pt_n[j])
        s[index] = np.dot(S[index, ], np.arange(k1+1))

        # 単語を生成
        word_id[index] = rmnom(psi[s[index], ], pt_n[j], v1, 0)
        word_long[index, j] = word_id[index, ]
        
    else:
        
        # 語順が2単語目の単語を生成
        # 多項分布からsyntaxを生成
        index = pt_list[j]
        res = rmnom(pi2[s[index-1], ], pt_n[j], k1+1, 1)
        S[index, ] = res[1]
        s[index] = res[0]

        # 単語履歴を更新
        for q in range(j):
            word_long[index, q] = word_long[index-1, q]
        if j < max_word:
            index_col = np.arange(j)
        else:
            index_col = np.arange(j-max_word, j)

        # attentionの単語を選択
        index_hmm = index[np.array(np.where(res[1][:, k1]==0)[0], dtype="int")]
        index_attention = index[np.array(np.where(res[1][:, k1]==1)[0], dtype="int")]        
        if len(index_attention) > 0:
            candidate_word = word_long[index_attention-1, ][:, index_col]
            logit = kappa[candidate_word, ]
            prob = np.exp(logit) / np.sum(np.exp(logit), axis=1)[:, np.newaxis]
            word = np.sum(candidate_word * rmnom(prob, prob.shape[0], prob.shape[1], 1)[1], axis=1)
            attention_id[index_attention] = word

        # attentionからトピックを生成
        res = rmnom(theta1[word, ], word.shape[0], k2, 1)
        Zi[index_attention, ] = res[1]
        z[index_attention] = res[0]

        # 単語を生成
        word_id[index_hmm] = rmnom(psi[s[index_hmm], ], index_hmm.shape[0], v1, 0)
        word_id[index_attention] = rmnom(phi[z[index_attention], ], index_attention.shape[0], v1, 0)
        word_long[index, j] = word_id[index]

In [237]:
attention_id[attention_id!=-1].shape

(607741,)

In [221]:
attention_id[attention_id!=-1].shape

(1022787,)

In [230]:
pd.concat((pd.DataFrame(attention_id), pd.DataFrame(word_long)), axis=1)

,0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70
0,-1,1193,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,-1,1193,266,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,-1,1193,266,1240,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,1240,1193,266,1240,827,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,1240,1193,266,1240,827,704,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
5,266,1193,266,1240,827,704,146,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
6,266,1193,266,1240,827,704,146,145,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
7,704,1193,266,1240,827,704,146,145,152,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
8,-1,1193,266,1240,827,704,146,145,152,79,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
9,152,1193,266,1240,827,704,146,145,152,79,208,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


array([], dtype=int32)

In [193]:
word_long[:, np.arange(2)]

array([[663,  -1],
       [663,  -1],
       [ -1,  -1],
       ...,
       [ -1,  -1],
       [ -1,  -1],
       [ -1,  -1]])

(30,)

In [174]:
j = 32

In [ ]:
def rmnom(pr, n, k, pattern):
    if pattern==1:
        z_id = np.array(np.argmax(np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis], axis=1), dtype="int")
        Z = np.diag(np.repeat(1, k))[z_id, ]
        return z_id, Z
    z_id = np.array(np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1), dtype="int")
    return z_id

In [48]:
psi.shape

(12, 1350)